In [ ]:
#Create a timeseries model that can predict concentrations of Pm2.5 in the atmosphere

In [1]:
## imports
import os
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import sklearn as skl
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
#path = os.path.join(os.getcwd(), 'Resources', 'all_city_data.csv')

In [3]:
df = pd.read_csv('all_city_data.csv').drop(columns = ['Unnamed: 0'])

In [4]:
df.head()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,dt
0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,2013-01-03 00:00:00
1,9.0,9.0,3.0,17.0,300.0,89.0,-0.5,1024.5,-21.4,0.0,2013-01-03 00:00:00
2,4.0,4.0,14.0,20.0,300.0,69.0,-0.7,1023.0,-18.8,0.0,2013-01-03 00:00:00
3,7.0,7.0,3.0,2.0,100.0,91.0,-2.3,1020.3,-20.7,0.0,2013-01-03 00:00:00
4,5.0,14.0,4.0,12.0,200.0,85.0,-0.5,1024.5,-21.4,0.0,2013-01-03 00:00:00


In [ ]:
# convert an array of values into a dataset matrix
# def create_dataset(dataset, look_back=1):
# 	dataX, dataY = [], []
# 	for i in range(len(dataset)-look_back-1):
# 		a = dataset[i:(i+look_back), 0]
# 		dataX.append(a)
# 		dataY.append(dataset[i + look_back, 0])
# 	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
# look_back = 1
# TrainX, TrainY = create_dataset(train, look_back)
# testX, testY = create_dataset(test, look_back)

In [ ]:

# reshape input to be [samples, time steps, features]
# trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [56]:
features = ["PM10", "SO2", "NO2", "CO", "O3", "TEMP", "PRES", "DEWP", "RAIN"]
scalar = StandardScaler()
# the fit_transform ops returns a 2d numpy.array, we cast it to a pd.DataFrame
standardized_features = pd.DataFrame(scalar.fit_transform(df[features].copy()), columns = features)
old_shape = df.shape
# drop the unnormalized features from the dataframe
df.drop(features, axis = 1, inplace = True)
# join back the normalized features
df = pd.concat([df, standardized_features], axis= 1)
assert old_shape == df.shape, "something went wrong!"

In [57]:
df.head()

,PM2.5,dt,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN
0,4.0,2013-01-03 00:00:00,-1.101254,-0.545907,-1.243556,-0.804110,0.348630,-1.242142,1.164563,-1.538262,-0.078906
1,9.0,2013-01-03 00:00:00,-1.046543,-0.592877,-0.958503,-0.804110,0.560363,-1.224639,1.308088,-1.726789,-0.078906
2,4.0,2013-01-03 00:00:00,-1.101254,-0.076203,-0.872987,-0.804110,0.207475,-1.242142,1.164563,-1.538262,-0.078906
3,7.0,2013-01-03 00:00:00,-1.068428,-0.592877,-1.386082,-0.976692,0.595652,-1.382164,0.906219,-1.676032,-0.078906
4,5.0,2013-01-03 00:00:00,-0.991832,-0.545907,-1.101029,-0.890401,0.489786,-1.224639,1.308088,-1.726789,-0.078906


In [58]:
time_index=pd.to_datetime(df['dt'])
df_kl=df_kl.set_index(time_index)
df_kl.index

DatetimeIndex(['2013-01-03 00:00:00', '2013-01-03 00:00:00',
               '2013-01-03 00:00:00', '2013-01-03 00:00:00',
               '2013-01-03 00:00:00', '2013-01-03 00:00:00',
               '2013-01-03 00:00:00', '2013-01-03 00:00:00',
               '2013-01-03 00:00:00', '2013-01-03 00:00:00',
               ...
               '2017-12-02 23:00:00', '2017-12-02 23:00:00',
               '2017-12-02 23:00:00', '2017-12-02 23:00:00',
               '2017-12-02 23:00:00', '2017-12-02 23:00:00',
               '2017-12-02 23:00:00', '2017-12-02 23:00:00',
               '2017-12-02 23:00:00', '2017-12-02 23:00:00'],
              dtype='datetime64[ns]', name='dt', length=383586, freq=None)

In [59]:
df_kl=df.drop('dt', axis=1)
y = df_kl['PM2.5']
X = df_kl.drop(columns= ['PM2.5'])
X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle=False)



In [2]:
# loaded_model = tf.keras.models.load_model("TEMP_test_model.h5")
# loaded_model.build()
# loaded_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 365, 50)           10400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_5 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 26        
Total params: 31,901
Trainable params: 31,901
Non-trainable params: 0
_________________________________________________________________


In [11]:
# pred_temp = loaded_model.predict(X_test)
# pred_temp

array([[-0.25250596],
       [-0.19487654],
       [-0.13481413],
       ...,
       [-0.29821187],
       [-0.3370747 ],
       [-0.3753718 ]], dtype=float32)

In [45]:

# model=tf.keras.models.Sequential()
# layer_1=tf.keras.layers.Dense(units=96, activation="tanh", input_dim=9)
# model.add(layer_1)
# layer_2=tf.keras.layers.Dense(units=80, activation="relu", input_dim=9)
# model.add(layer_2)
# layer_3=tf.keras.layers.Dense(units=64, activation="relu", input_dim=9)
# model.add(layer_3)
# layer_4=tf.keras.layers.Dense(units=48, activation="relu", input_dim=9)
# #model.add(layer_4)
# layer_5=tf.keras.layers.Dense(units=32, activation="relu", input_dim=9)
# #model.add(layer_5)
# output_layer=tf.keras.layers.Dense(units=1)
# model.add(output_layer)
# model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 96)                960       
_________________________________________________________________
dense_51 (Dense)             (None, 80)                7760      
_________________________________________________________________
dense_52 (Dense)             (None, 64)                5184      
_________________________________________________________________
dense_55 (Dense)             (None, 1)                 65        
Total params: 13,969
Trainable params: 13,969
Non-trainable params: 0
_________________________________________________________________


In [47]:
scores = model.evaluate(X_test, y_test)
print(model.metrics_names[0], scores)

2997/2997 [==============================] - 4s 1ms/step - loss: 526.6973 - mean_absolute_error: 13.9873
loss [526.6973266601562, 13.987262725830078]


In [60]:
def create_model(hp):
    model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=9))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    model.compile(loss="mean_squared_error", optimizer='adam', metrics=["mean_squared_error"])
    
    return model

In [ ]:
pip install keras-tuner --upgrade

In [61]:
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="mean_squared_error",
    max_epochs=20,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [62]:
tuner.search(X_train,y_train,epochs=20,validation_data=(X_test,y_test))

Trial 45 Complete [00h 06m 22s]
mean_squared_error: 12609.1201171875

Best mean_squared_error So Far: 12609.1201171875
Total elapsed time: 02h 16m 52s
INFO:tensorflow:Oracle triggered exit


In [63]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 3,
 'num_layers': 3,
 'tuner/bracket': 0,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_0': 5,
 'units_1': 7,
 'units_2': 5,
 'units_3': 5,
 'units_4': 3,
 'units_5': 7}

In [64]:
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2997/2997 - 3s - loss: 12568.2422 - mean_squared_error: 12568.2422
Loss: 12568.2421875, Accuracy: 12568.2421875


In [65]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
8991/8991 [==============================] - 17s 2ms/step - loss: 345.5502 - mean_absolute_error: 12.1158
Epoch 2/100
3352/8991 [==========>...................] - ETA: 10s - loss: 336.9549 - mean_absolute_error: 12.0217

KeyboardInterrupt: ignored